In [1]:
import numpy as np
import pandas as pd
import open3d as o3d
import transforms3d as tf3d
from scipy.spatial.transform import Rotation as R
import csv
import cv2
from pyntcloud import PyntCloud as pt

import os
from tqdm import tqdm

Perspective projection matrix

In [2]:
cx = 6.960217e+02
# cy = 2.241806e+02
cy = 275
# cx = 696
fx = 9.597910e+02*1.5
fy = 9.599251e+02*1.5
# fx = 960
# fy = 960
K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
dist_coeff = np.array([-3.691481e-01, 1.968681e-01, 1.353473e-03, 5.677587e-04,-6.770705e-02])

In [3]:
def clean_data(data):
        data['SimTime'] = np.round(data['SimTime'], 2)
        data.drop('WallTime', inplace=True, axis=1)
        data.drop('PropertyType', inplace=True, axis=1)
        data.drop('InstanceType', inplace=True, axis=1)
        data.drop('ModelInstanceIdPath', inplace=True, axis=1)
        index_to_keep = data[data['SimTime'] >= 6.0].index.min()

        # Drop rows before the index_to_keep
        if index_to_keep is not None:
            data = data.iloc[index_to_keep:]
        return data
    

In [4]:
def read_data(gt_path):
    data = pd.DataFrame()
    gt_data_folders = ['gt100.csv','gt101.csv','gt102.csv','gt103.csv','gt104.csv','gt105.csv','gt106.csv','gt107.csv','gt108.csv','gt109.csv','gt110.csv','gt111.csv','gt112.csv','gt113.csv','gt114.csv','gt115.csv','gt116.csv','gt117.csv','gt118.csv','gt119.csv','gt120.csv']
    for file_name in gt_data_folders:
        gt_filepath = os.path.join(gt_path, file_name)
        gt_data = pd.read_csv(gt_filepath)
        gt_data =clean_data(gt_data)
        data = pd.concat([data, gt_data], ignore_index=True)
    return data
        

In [6]:
# image = cv2.imread("D:\\Thesis\\test\\color_no_mag_90\\color50500.png")
pc_read = o3d.io.read_point_cloud("D:\\master_thesis\\pc_1\\pcd100\\output.pcd_6000_60.pcd")
# pcd_data = np.asarray(pc_read.points)

# data = pd.read_csv("D:\\\master_thesis\\gt_data\\gt101.csv")
# # data_tf = pd.read_csv("D:\\Thesis\\test\\tf_90_104.csv")
# data['SimTime'] = np.round(data['SimTime'], 2)
# data.drop('WallTime', inplace=True, axis=1)
# data.drop('PropertyType', inplace=True, axis=1)
# data.drop('InstanceType', inplace=True, axis=1)
# data.drop('ModelInstanceIdPath', inplace=True, axis=1)
# index_to_keep = data[data['SimTime'] >= 6.0].index.min()

# # Drop rows before the index_to_keep
# if index_to_keep is not None:
#     data = data.iloc[index_to_keep:]
    # data_tf = data_tf.iloc[index_to_keep:]

(2880, 4)

In [91]:
# gt_path = "D:\\master_thesis\\gt_data"
# data = read_data(gt_path)

In [92]:
def data_array_from_column(df, column_name):
    vectors = df[column_name].apply(eval)
    vectors_df = pd.DataFrame(vectors.tolist())
   

    return np.asarray(vectors_df)

In [93]:
childsizes_all = data.loc[data['PropertyName']=='childsizes']
childsizes_array = data_array_from_column(childsizes_all,'Data')
childpositions_all = data.loc[data['PropertyName']=='childpositions']
childpositions_array = data_array_from_column(childpositions_all,'Data')
childorientations_all = data.loc[data['PropertyName']=='childorientations']
childorientations_array = data_array_from_column(childorientations_all,'Data')


In [94]:
childsizes_filtered = []
childpositions_filtered = []
childorientations_filtered = []
for i in range(childsizes_array.shape[0]):
        childsizes_array_temp = [x for x in childsizes_array[i] if x is not None]
        childpositions_array_temp = [x for x in childpositions_array[i] if x is not None]   
        childorientations_array_temp = [x for x in childorientations_array[i] if x is not None]
        childpositions_filtered.append(childpositions_array_temp)
        childsizes_filtered.append(childsizes_array_temp) 
        childorientations_filtered.append(childorientations_array_temp)

In [95]:
BB_center = []
BB_orientation = []
BB_length = []
BB_width = []
BB_height = []
# BB_dimensions = []

for i in range(len(childsizes_filtered)):
    for j in range(len(childsizes_filtered[i])):
        length = (childsizes_filtered[i][j][0])*0.8
        width = (childsizes_filtered[i][j][1])*0.67
        height = (childsizes_filtered[i][j][2])*0.5
        # if length <1 and width <1 and height <1:
        #     length, width, height = 5.0,2.2,2.4
        # else:
        #     length, width, height = length, width, height

        center = childpositions_filtered[i][j]
        orientation = childorientations_filtered[i][j][0]
        
        BB_length.append(length)
        BB_width.append(width)
        BB_height.append(height)
        # BB_dimensions.append([length, width, height])
        BB_center.append(center)
        BB_orientation.append(orientation)



In [96]:

total_objects = len(BB_center)
total_frames = int(len(childsizes_array)) # 6 is the number of times childsize is calculated in one time frame
total_unique_objects_per_frame = int(total_objects/total_frames)

In [97]:
total_objects, total_frames, total_unique_objects_per_frame

(9216, 576, 16)

In [98]:
BB_center = np.reshape(BB_center, (total_frames, total_unique_objects_per_frame, 3))
BB_length = np.reshape(BB_length, (total_frames, total_unique_objects_per_frame))
BB_width = np.reshape(BB_width, (total_frames, total_unique_objects_per_frame))
BB_height = np.reshape(BB_height, (total_frames, total_unique_objects_per_frame))
BB_orientation = np.reshape(BB_orientation, (total_frames, total_unique_objects_per_frame))
# BB_dimensions = np.reshape(BB_dimensions, (total_frames, total_unique_objects_per_frame, 3))

In [99]:
# tf_camera_data = data.loc[data['InstanceName']=='Simulated Front Camera (internal reference frame)']
tf_camera_data = data.loc[data['InstanceName']=='FrontCamera CCS']
tf_camera_array = data_array_from_column(tf_camera_data,'Data').reshape(-1,4,4)

tf_lidar_data = data.loc[data['InstanceName']=='Velodyne HDL-64E']
tf_lidar_array = data_array_from_column(tf_lidar_data,'Data').reshape(-1,4,4)

In [100]:
import math
camera_projection_array = []
# Define camera configuration
fov = 51.6  # Field of View (degrees)
aspect_ratio = 1392/512  # Widescreen aspect ratio
near_plane = 0.01  # Near plane distance
far_plane = 500  # Far plane distance
camera_position = (0, 2, 5)  # Camera position in world space

# Function to calculate tangent of half FOV (common for perspective projection matrix)
def tan_half_fov(fov):
  radians = math.radians(fov / 2)
  return math.tan( radians)

# Create an empty perspective projection matrix (replace with library function in practice)
camera_projection_matrix = [[0 for _ in range(4)] for _ in range(4)]

# Fill upper left 3x3 sub-matrix (common perspective projection structure)
half_fov_tan = tan_half_fov(fov)
camera_projection_matrix[0][0] = 1 / (aspect_ratio * half_fov_tan)
camera_projection_matrix[1][1] = 1 / half_fov_tan
camera_projection_matrix[2][2] = (near_plane + far_plane) / (near_plane - far_plane)

# Fill other elements based on perspective projection formula (replace with library function in practice)
camera_projection_matrix[2][3] = 2 * near_plane * far_plane / (near_plane - far_plane)
camera_projection_matrix[3][2] = -1

In [101]:
V2C_array = []
inv_lidar_tf_array = []
for i in range(len(tf_camera_array)):
    inv_lidar_tf = np.linalg.inv(tf_lidar_array[i])
    V2C = np.dot(inv_lidar_tf,tf_camera_array[i])
    flip_tf = np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
    V2C = np.dot(flip_tf,V2C)

    V2C_array.append(V2C)
    inv_lidar_tf_array.append(inv_lidar_tf)


In [102]:
# position_from_camera = []
position_from_lidar = []
for i in tqdm(range(total_frames)):
    for j in range(total_unique_objects_per_frame):
        world_pts = np.reshape(BB_center[i][j][:3],(3,1))
        Rot_world_lidar = inv_lidar_tf_array[i][:3,:3]
        trans_world_lidar = np.reshape(inv_lidar_tf_array[i][:3,3],(3,1))
        rotated_lidar = np.dot(Rot_world_lidar,world_pts)
        pos_from_lidar = rotated_lidar + trans_world_lidar
        # dist_lidar_camera = V2C_array[i][:3,3]
        # pos_from_camera = [pos_from_lidar[0]-dist_lidar_camera[0], pos_from_lidar[1]-dist_lidar_camera[1], pos_from_lidar[2]-dist_lidar_camera[2]]
        # Rot_lidar_camera = V2C_array[i][:3,:3]
        # trans_lidar_camera = np.reshape(V2C_array[i][:3,3],(3,1))
        # rotated_camera = np.dot(Rot_lidar_camera,pos_from_lidar)
        # pos_from_camera = rotated_camera+trans_lidar_camera
        position_from_lidar.append(pos_from_lidar)
        # position_from_camera.append(pos_from_camera)
        
    
# position_from_camera = np.reshape(position_from_camera, (total_frames, total_unique_objects_per_frame, 3))    
position_from_lidar = np.reshape(position_from_lidar, (total_frames, total_unique_objects_per_frame, 3))

100%|██████████| 576/576 [00:00<00:00, 4027.81it/s]


In [105]:
position_from_camera = []
Rt = []
# posittion_from_lidar2 = []
for i in range(total_frames):
    for j in range(total_unique_objects_per_frame):
        positionB = np.reshape(BB_center[i][j][:3],(3,1))
        TF_CB = np.linalg.inv(tf_camera_array[i].reshape(4,4))
        RotCB = TF_CB[:3,:3]
        translationCB = np.reshape(TF_CB[:3,3],(3,1)) + np.reshape([0,0.5,1.0],(3,1))
        R_T = np.hstack((RotCB,translationCB))
        rot = np.dot(RotCB,positionB)
        pos = rot + translationCB
        # pos_homogeneous = np.vstack((pos,1))
        # C2V = np.linalg.inv(V2C_array[i])
        # pos_lidar_homogeneous = np.dot(C2V,pos_homogeneous)
        # pos_lidar = pos_lidar_homogeneous[:3]
        position_from_camera.append(pos)
        Rt.append(R_T)
        # posittion_from_lidar2.append(pos_lidar)
    
    
position_from_camera = np.reshape(position_from_camera, (total_frames, total_unique_objects_per_frame, 3))
Rt = np.reshape(Rt,(total_frames,total_unique_objects_per_frame,3,4))
# posittion_from_lidar2 = np.reshape(posittion_from_lidar2, (total_frames, total_unique_objects_per_frame, 3))

In [ ]:
Rt

In [62]:
W2V = tf_lidar_array[445]
W2C = tf_camera_array[445]
V2W = np.linalg.inv(W2V)
flip_tf = np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])


In [63]:
V2C = flip_tf@V2W@W2C
C2V = np.linalg.inv(V2C)

In [64]:
vc = V2C_array[445]
cv = np.linalg.inv(vc)

In [65]:
C2V[:3,:3]@position_from_camera[445][7],V2C[:3,:3]@position_from_lidar[445][7]

(array([-9.17417916e-03,  2.41397757e+01, -9.11517660e-01]),
 array([-9.56326496e-03,  8.07518216e-01,  2.39536817e+01]))

In [66]:
C2V[:3,:3]@position_from_camera[445][7]+C2V[:3,3]

array([-9.44480859e-03,  2.37437757e+01, -1.72542397e+00])

In [67]:
position_from_lidar[445][7] , position_from_camera[445][7]

(array([-9.58722450e-03,  2.39536817e+01, -8.07517932e-01]),
 array([-9.14713388e-03,  9.11517932e-01,  2.41397757e+01]))

In [68]:
rotation_y = []
for i in range((total_frames)):
    for j in range((total_unique_objects_per_frame)):
        yaw_vehicle = np.rad2deg(BB_orientation[i][j])
        r = R.from_matrix(tf_camera_array[i][:3,:3])
        roll, pitch, yaw = (r.as_euler('xyz', degrees=True))
        angle_diff = np.deg2rad((yaw+90.0)-yaw_vehicle)
        rotation_y.append(angle_diff)
rotation_y = np.reshape(rotation_y, (total_frames, total_unique_objects_per_frame))

BB_lidar


In [69]:
filtered_bb_index = []

Lidar_clipping_range = [5, -20, -3, 40, 20, 1]
for i in range(total_frames):
    temp_list = []
    for idx, j in enumerate(position_from_lidar[i]):
        if j[0] < Lidar_clipping_range[4] and j[0] > Lidar_clipping_range[1] and j[1] < Lidar_clipping_range[3] and j[1] > Lidar_clipping_range[0] and j[2] < Lidar_clipping_range[5] and j[2] > Lidar_clipping_range[2]:
            temp_list.append(idx)
    filtered_bb_index.append(temp_list)

In [70]:
# BB_coordinates_lidar = []
# for i in range((total_frames)):
#   for j in range((total_unique_objects_per_frame)):
#         x = position_from_lidar[i][j][0]
#         y = position_from_lidar[i][j][1]
#         z = position_from_lidar[i][j][2]

#         l = BB_length[i][j]
#         h = BB_height[i][j]
#         w = BB_width[i][j]

#         theta = rotation_y[i][j]
#         Rot_matrix_bb = tf3d.euler.euler2mat(0,0,theta)

#         edges = []
#         oriented_edges = []
#         e1 = (x-w,y-l,z-h)
#         e2 = (x-w,y-l,z+h)
#         e3 = (x-w,y+l,z+h)
#         e4 = (x-w,y+l,z-h)
#         e5 = (x+w,y-l,z-h)
#         e6 = (x+w,y-l,z+h)
#         e7 = (x+w,y+l,z+h)
#         e8 = (x+w,y+l,z-h)
#         edges.append(e1)
#         edges.append(e2)
#         edges.append(e3)
#         edges.append(e4)
#         edges.append(e5)
#         edges.append(e6)
#         edges.append(e7)
#         edges.append(e8)
#         # BB_coordinates_lidar.append(edges)
#         for edge in edges:
#             edge = np.dot(Rot_matrix_bb, edge)
#             BB_coordinates_lidar.append(edge)

# BB_coordinates_lidar = np.reshape( BB_coordinates_lidar,(total_frames,total_unique_objects_per_frame,8,3))

BB_camera


In [71]:
BB_coordinates_camera = []
# BB_coordinates_lidar = []
for i in range((total_frames)):
  for j in range((total_unique_objects_per_frame)):
        l, h, w = BB_length[i][j], BB_height[i][j], BB_width[i][j]
        x_corners = [w / 2, w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2]
        # y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
        y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2]
        z_corners = [l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2]

        R = np.array([[np.cos(rotation_y[i][j]), 0, np.sin(rotation_y[i][j])],
                        [0, 1, 0],
                        [-np.sin(rotation_y[i][j]), 0, np.cos(rotation_y[i][j])]])
        corners3d = np.vstack([x_corners, y_corners, z_corners])  # (3, 8)
        corners3d = np.dot(R, corners3d).T
        corners3d = corners3d + position_from_camera[i][j]
        C2V = np.linalg.inv(V2C_array[i])
        corners3d_lidar = np.dot(C2V[:3, :3], corners3d.T) + C2V[:3, 3].reshape(-1, 1)
        BB_coordinates_camera.append(corners3d)
        
BB_coordinates_camera = np.reshape( BB_coordinates_camera,(total_frames,total_unique_objects_per_frame,8,3))

        

BB coordinates LIDAR

In [72]:
# BB_coordinates_lidar = []
# for i in range((total_frames)):
#   for j in range((total_unique_objects_per_frame)):
#     l, h, w = (BB_length[i][j])/0.67, (BB_height[i][j])/0.67, (BB_width[i][j])/0.67
#     x_corners = [w / 2, w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2]
#     # y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
#     y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2]
#     z_corners = [l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2]

#     R = np.array([[np.cos(rotation_y[i][j]), 0, np.sin(rotation_y[i][j])],
#                     [0, 1, 0],
#                     [-np.sin(rotation_y[i][j]), 0, np.cos(rotation_y[i][j])]])
#     corners3d = np.vstack([x_corners, y_corners, z_corners])  # (3, 8)
#     corners3d = np.dot(R, corners3d).T
#     C2V = np.linalg.inv(V2C_array[i])
#     position_camera_homogeneous = np.array([position_from_camera[i][j][0], position_from_camera[i][j][1], position_from_camera[i][j][2], 1])
#     position_lidar_homogeneous = np.dot(C2V, position_camera_homogeneous)
#     position_from_lidar = position_lidar_homogeneous[:3]
#     corners3d_lidar = corners3d + position_from_lidar
        
        
#     BB_coordinates_lidar.append(corners3d_lidar)
# BB_coordinates_lidar = np.reshape( BB_coordinates_lidar,(total_frames,total_unique_objects_per_frame,8,3))

fov calcu

In [73]:
p = 4.5*10-3
w = p*1392
f = fx*p
fov = np.rad2deg(2*np.arctan(w/(2*f)))
fov

51.60182925042597

CAMERA BB

In [74]:
image = cv2.undistort(image, K, dist_coeff)

NameError: name 'image' is not defined

In [30]:
# image_bb_corners = []
# for i in range(total_frames):
#     for j in range(total_unique_objects_per_frame):
#         for k in range(len(BB_coordinates_camera[i][j])):
#             translated_point = BB_coordinates_camera[i][j][k]
#             homogenous_3d_point = np.array([translated_point[0], translated_point[1], translated_point[2], 1])
#             clip_pt = np.dot(camera_projection_matrix, homogenous_3d_point.T)
#             normalized_device_coordinates = clip_pt / clip_pt[3]
#             screen_width = 1392
#             screen_height = 512
#             image_x = int((normalized_device_coordinates[0] + 1) * screen_width / 2)
#             image_y = int((1 - normalized_device_coordinates[1]) * screen_height / 2)
#             # proj_pt = K@translated_point.T
#             # point_2d = (proj_pt[:-1] / proj_pt[-1]).astype(int)
#             point_2d = (image_x, image_y)
#             image_bb_corners.append(point_2d)
# image_bb_corners = np.reshape(image_bb_corners,(total_frames,total_unique_objects_per_frame,8,2))

In [31]:
def generate_truncation(position_from_camera, BB_length, BB_width, BB_height,K, BB_orientation):
    truncation = []
    for i in range(filtered_bb_index):
        temp_list = []
        for j in range(filtered_bb_index[i]):
            center = position_from_camera[i][j]
            l = BB_length[i][j]*0.8
            w = BB_width[i][j]*0.67
            h = BB_height[i][j]*0.5
            if BB_orientation[i][j] >-0.7853 and BB_orientation[i][j] < 0.7853:
                car_left_end = [center[0] + w/2, center[1], center[2]] # sideways
                car_right_end = [center[0] - w/2, center[1], center[2]]
            else:
                car_left_end = [center[0] + l/2, center[1], center[2]]
                car_right_end = [center[0] - l/2, center[1], center[2]]
            car_bottom = [center[0], center[1]-h/2, center[2]] # infront
            car_roof = [center[0], center[1]+h/2, center[2]]
            # car_right = [center[0] + w/2, center[1], center[2]]
            # car_left = [center[0] - w/2, center[1], center[2]]
            proj_pt = K@center.T
            proj_right_end = K@car_right_end.T
            proj_left_end = K@car_left_end.T
            proj_roof = K@car_roof.T
            proj_bottom = K@car_bottom.T
            # proj_right = K@car_right.T
            
            u,v = (proj_pt[:-1] / proj_pt[-1]).astype(int)
            p,_ = (proj_right_end[:-1] / proj_right_end[-1]).astype(int)
            q,_ = (proj_left_end[:-1] / proj_left_end[-1]).astype(int)
            _,n = (proj_bottom[:-1] / proj_bottom[-1]).astype(int)
            _,m = (proj_roof[:-1] / proj_roof[-1]).astype(int)
            proj_l = (p-u)*2
            # proj_w = (s-u)*2
            proj_h = (n -v)*2
            if p < 1392 and q > 0  and n < 512:
                trunc = 0
            elif q >= 1392 or p <= 0 or m>=512 :
                trunc = 1
            elif q<1392 and p>1392:
                trunc = (p - 1392)/proj_l
            elif q < 0 and p > 0:
                trunc = (0-q)/proj_l
            elif n > 512 and m < 512:
                trunc = (n-512)/proj_h

            temp_list.append(trunc)
        truncation.append(temp_list)
            
    return truncation

In [32]:
image_bb_corners = []
for i in range(total_frames):
    for j in range(total_unique_objects_per_frame):
        for k in range(len(BB_coordinates_camera[i][j])):
            translated_point = BB_coordinates_camera[i][j][k]
            # new_point = np.array([translated_point[0],translated_point[2],translated_point[1]])
            # proj_pt = K@new_point.T
            proj_pt = K@translated_point.T
            point_2d = (proj_pt[:-1] / proj_pt[-1]).astype(int)
            image_bb_corners.append(point_2d)
image_bb_corners = np.reshape(image_bb_corners,(total_frames,total_unique_objects_per_frame,8,2))
   

In [33]:
# position_from_camera[445][8], position_from_camera[445][7]

In [34]:
# BB_coordinates_camera[445][8], BB_coordinates_camera[445][7]

In [35]:
test_img_bb = image_bb_corners[445][7]
test_lidar_bb = BB_coordinates_lidar[445][7]

In [36]:
max_x = np.max(test_img_bb[:, 0])
min_x = np.min(test_img_bb[:, 0])
max_y = np.max(test_img_bb[:, 1])
min_y = np.min(test_img_bb[:, 1])

In [37]:
max_x, min_x, max_y, min_y

(806, 647, 409, 237)

In [38]:
cv2.rectangle(image, (min_x, min_y), (max_x, max_y), (255, 255, 255), 2)
# cv2.rectangle(image, (min_x, ymin), (max_x, ymax), (0, 0,0), 2)
# 
# Display the image with the 2D bounding box
cv2.imshow("2D Bounding Box", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [39]:
pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pcd_data)
o3d.io.write_point_cloud("pcd1.ply",pcd1)

pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(test_lidar_bb)
o3d.io.write_point_cloud("pcd2.ply",pcd2)

pcd1_read = o3d.io.read_point_cloud("pcd1.ply")
pcd2_read = o3d.io.read_point_cloud("pcd2.ply")

pcd1_read.paint_uniform_color([0,0,0.75]) # choose a unique rgb value for each point cloud
pcd2_read.paint_uniform_color([0,0.5,0])

bb_data = o3d.geometry.AxisAlignedBoundingBox.create_from_points(pcd2.points)
obb_data = o3d.geometry.AxisAlignedBoundingBox.get_oriented_bounding_box(bb_data)
# o3d.visualization.draw_geometries([pcd1_read,pcd2_read])
o3d.visualization.draw_geometries([pcd1_read,obb_data])

In [40]:
color = (0, 255, 0)  # Green color for the bounding box (BGR format)
thickness = 2  # Thickness of the bounding box lines
cv2.line(image, test_img_bb[0], test_img_bb[1], color, thickness)
cv2.line(image, test_img_bb[1], test_img_bb[2], color, thickness)
cv2.line(image, test_img_bb[2], test_img_bb[3], color, thickness)
cv2.line(image, test_img_bb[3], test_img_bb[0], color, thickness)
cv2.line(image, test_img_bb[4], test_img_bb[5], color, thickness)
cv2.line(image, test_img_bb[5], test_img_bb[6], color, thickness)
cv2.line(image, test_img_bb[6], test_img_bb[7], color, thickness)
cv2.line(image, test_img_bb[7], test_img_bb[4], color, thickness)
cv2.line(image, test_img_bb[0], test_img_bb[4], color, thickness)
cv2.line(image, test_img_bb[1], test_img_bb[5], color, thickness)
cv2.line(image, test_img_bb[2], test_img_bb[6], color, thickness)
cv2.line(image, test_img_bb[3], test_img_bb[7], color, thickness)

# Display the image with the bounding box
cv2.imshow('Bounding Box', image)
cv2.waitKey(0)
cv2.destroyAllWindows()